In [2]:
load('../src/sccc_sugiyama.sage')

In [3]:
# Define the finite field and polynomial ring
F = GF(8, 'a')  # Finite field of size 8
a = F.gen()  # Primitive element generator of F8
R = PolynomialRing(F,'t')  # Polynomial Ring on t over F
K.<t> = FractionField(R)  # field of rational functions over F, F(t)

print("K:", K)

# Define the automorphism and its inverse
sigma = K.hom([1/(t+a)])
inverse_sigma = K.hom([(1-a*t)/t])

# Define the ore extension F(t)[x;sigma]
P.<x> = SkewPolynomialRing(K, sigma) ; P

K: Fraction Field of Univariate Polynomial Ring in t over Finite Field in a of size 2^3


Ore Polynomial Ring in x over Fraction Field of Univariate Polynomial Ring in t over Finite Field in a of size 2^3 twisted by t |--> 1/(t + a)

Calculamos el orden del automorfismo, es decir el primer $n > 0$ tal que $\sigma^n(t) = t$

In [4]:
# Check the Automorphism order
order_of_automorphism(K, sigma)

9

Obtenemos que el orden de $\sigma$ es 9, por tanto trabajaremos en $$\mathcal{R} = \frac{\mathbb{F}(t)[x;\sigma]}{\langle x^n -1\rangle}$$ con $n = 9$.

Ahora debemos buscar un $\alpha$ de manera que $\{\alpha,\sigma(\alpha),\dots,\sigma^{n-1}(\alpha)\}$ sea una base normal de la extensión de cuerpos $\mathbb{F}(t)^{\sigma} \subseteq \mathbb{F}(t)$, esto es equivalente a que el siguiente determinante


\begin{vmatrix}
\alpha & \sigma(\alpha) & \sigma^2(\alpha) & \cdots & \sigma^{n-1}(\alpha) \\
\sigma(\alpha) & \sigma^2(\alpha) & \sigma^3(\alpha) & \cdots & \alpha\\
\vdots & \vdots & \vdots & \ddots & \vdots \\
\sigma^{n-1}(\alpha) & c & \sigma(\alpha) & \cdots & \sigma^{n-2}(\alpha)
\end{vmatrix}

sea distinto de cero.

In [5]:
alpha = t
normal_basis(sigma,alpha,K)

True

Como forma una base normal podemos seguir con la construcción de un código convolucional sesgado RS.
Ahora debemos calcular $\beta = \alpha^{-1}\sigma(\alpha)$.

In [6]:
beta = alpha^(-1)*sigma(alpha) ; beta


1/(t^2 + a*t)

Sabemos que el mínimo común múltiplo por la izquierda de $g = \left[ \{x - \sigma^i(\beta)\}_{i=0,1,2,3,4} \right]_\ell$ genera un $[9,5]$-código convolucional sesgado Reed-Solomon de distancia diseñada $\delta = 6$.

## Construcción de un código convolucional sesgado RS

In [7]:
g_list = []
for i in range(4):
    sigma_i = sigma^i
    g_list.append(x - sigma_i(beta))
g_list

[x + 1/(t^2 + a*t),
 x + ((a^2 + 1)*t^2 + a)/(t + a^2 + a + 1),
 x + ((a + 1)*t^2 + a^2 + 1)/(t^2 + a^2*t + a^2 + a + 1),
 x + (a*t^2 + a^2)/(t^2 + (a^2 + a)*t + a^2 + a + 1)]

Construimos $\mathcal{C}$ a partir de los elementos anteriores. También necesitamos haber definido previamente el automorfismo inverso si queremos decodificar.

In [20]:
C = SkewRSConvolutionalCode(roots = g_list,inverse_aut = inverse_sigma,alpha = alpha) ; C

[9,5] Skew Reed-Solomon Convolutional Code on Ore Polynomial Ring in x over Fraction Field of Univariate Polynomial Ring in t over Finite Field in a of size 2^3 twisted by t |--> 1/(t + a) with designed distance 5

A partir de $\mathcal{C}$ podemos obtener varios elementos.

**Polinomio generador del código**

In [21]:
C.generator_polynomial()

x^4 + (((a^2 + 1)*t^5 + (a + 1)*t^4 + a*t + a^2 + a + 1)/(t^5 + (a^2 + 1)*t^4 + (a^2 + a + 1)*t + a^2 + a))*x^3 + (((a + 1)*t^6 + (a + 1)*t^5 + (a^2 + a + 1)*t^4 + t^2 + t + a^2)/(t^6 + t^5 + a*t^4 + (a^2 + a + 1)*t^2 + (a^2 + a + 1)*t + a^2 + 1))*x^2 + ((a^2*t^7 + (a^2 + a + 1)*t^6 + (a^2 + 1)*t^5 + (a^2 + a + 1)*t^4)/(t^7 + a*t^6 + t^5 + (a^2 + a)*t^4 + (a^2 + a + 1)*t^3 + (a^2 + 1)*t^2 + (a^2 + a + 1)*t + a^2))*x + ((a^2 + 1)*t^6 + (a^2 + a)*t^5 + (a^2 + a)*t^4 + t^3 + (a^2 + 1)*t^2 + t + a)/(t^7 + (a^2 + a + 1)*t^6 + a*t^5 + a^2*t^4 + (a^2 + a + 1)*t^3 + (a + 1)*t^2 + (a^2 + 1)*t + 1)

**Longitud del Código**

In [22]:
C.length()

9

**Dimensión del Código**

In [23]:
C.dimension()

5

## Codificación de palabras código

In [24]:
E = SkewRSConvolutionalEncoder(C) ; E

SkewRSConvolutionalEncoder for a [9,5] Skew Reed-Solomon Convolutional Code on Ore Polynomial Ring in x over Fraction Field of Univariate Polynomial Ring in t over Finite Field in a of size 2^3 twisted by t |--> 1/(t + a) with designed distance 5

Podemos probar a codificar varios mensajes. Como la dimensión del código es $k = 5$ estos mensajes deberán ser de grado menor o igual que $4$. Recordemos que el espacio de estos mensajes es $\mathbb{F}(t)[x;\sigma]$.

Codificación del mensaje $m = \frac{at^4 + at^2 + t + a}{t^3 + a}x^4 + \frac{t^2 + at + a + 1}{t + a^2}x + \frac{t + a}{t^5 + a^2}$

In [25]:
m = (a*t^4 + a*t^2 + t + a)/(t^3 + a) * x^4 + (t^2 + a*t + a^3)/(t + a^2) * x + (t + a)/(t^5 + a^2) ; m

((a*t^4 + a*t^2 + t + a)/(t^3 + a))*x^4 + ((t^2 + a*t + a + 1)/(t + a^2))*x + (t + a)/(t^5 + a^2)

In [26]:
c = E.encode(m) ; c

((a*t^4 + a*t^2 + t + a)/(t^3 + a))*x^8 + ((a*t^9 + t^8 + a*t^7 + t^5 + a*t^4 + (a^2 + a)*t^3 + a*t + a + 1)/(t^4 + (a^2 + a + 1)*t^3 + a*t + a^2 + 1))*x^7 + (((a^2 + 1)*t^10 + (a^2 + 1)*t^9 + a^2*t^8 + a*t^7 + (a^2 + 1)*t^6 + (a^2 + a)*t^5 + (a^2 + 1)*t^3 + a*t^2 + a^2 + a + 1)/(t^5 + (a^2 + 1)*t^4 + (a^2 + 1)*t^3 + a*t^2 + t + 1))*x^6 + ((a^2*t^12 + (a^2 + 1)*t^11 + (a^2 + a + 1)*t^10 + a^2*t^9 + a^2*t^8 + t^7 + (a + 1)*t^6 + (a + 1)*t^5 + (a^2 + a)*t^4 + a*t^3 + (a^2 + a + 1)*t^2 + a*t)/(t^6 + t^5 + (a^2 + a + 1)*t^4 + a*t^2 + (a^2 + 1)*t + a^2))*x^5 + (((a^2 + 1)*t^21 + (a^2 + a + 1)*t^20 + a*t^19 + (a^2 + 1)*t^18 + t^17 + (a^2 + a + 1)*t^16 + a*t^15 + (a^2 + 1)*t^14 + (a^2 + a)*t^12 + (a^2 + a)*t^11 + (a^2 + 1)*t^10 + (a^2 + 1)*t^9 + (a^2 + a)*t^8 + t^6 + t^5 + t^4 + (a^2 + a)*t^3 + (a^2 + a)*t^2 + (a^2 + 1)*t + 1)/(t^15 + (a + 1)*t^14 + t^13 + a*t^12 + a^2*t^11 + (a^2 + 1)*t^9 + a^2*t^7 + a*t^6 + (a^2 + a)*t^5 + (a + 1)*t^4 + (a^2 + a)*t^3 + t^2 + (a^2 + a)*t))*x^4 + (((a + 1)*t^

Al igual que hemos codificado el mensaje podemos hacer el **proceso inverso**, obteniendo el mensaje original a partir del mensaje codificado.

In [27]:
m2 = E.unencode(c) ; m2

((a*t^4 + a*t^2 + t + a)/(t^3 + a))*x^4 + ((t^2 + a*t + a + 1)/(t + a^2))*x + (t + a)/(t^5 + a^2)

Comprobamos que efectivamente `m2` coincide con el mensaje original.

In [28]:
m2 == m

True

## Decodificación de mensajes

In [29]:
D = SkewRSSugiyamaDecoder(C) ; D

<A Sugiyama-like algorithm for [9,5] Skew Reed-Solomon Convolutional Code on Ore Polynomial Ring in x over Fraction Field of Univariate Polynomial Ring in t over Finite Field in a of size 2^3 twisted by t |--> 1/(t + a) with designed distance 5 with correction capability of 2 errors>

Supongamos que recibimos el siguiente mensaje

In [30]:
y = x^4 + (((a^2 + 1)*t^5 + (a + 1)*t^4 + a*t + a^2 + a + 1)/(t^5 + (a^2 + 1)*t^4 + (a^2 + a + 1)*t + a^2 + a))*x^3 + (((a + 1)*t^6 + (a + 1)*t^5 + (a^2 + a + 1)*t^4 + t^2 + t + a^2)/(t^6 + t^5 + a*t^4 + (a^2 + a + 1)*t^2 + (a^2 + a + 1)*t + a^2 + 1))*x^2 ; y

x^4 + (((a^2 + 1)*t^5 + (a + 1)*t^4 + a*t + a^2 + a + 1)/(t^5 + (a^2 + 1)*t^4 + (a^2 + a + 1)*t + a^2 + a))*x^3 + (((a + 1)*t^6 + (a + 1)*t^5 + (a^2 + a + 1)*t^4 + t^2 + t + a^2)/(t^6 + t^5 + a*t^4 + (a^2 + a + 1)*t^2 + (a^2 + a + 1)*t + a^2 + 1))*x^2

Este mensaje es simplemente el **polinomio generador del código** (que obviamente también es una palabra código válida) sin los términos de grado 0 y 1, por tanto el código debería ser capaz de reconstruirlo correctamente, ya que tiene una capacidad de corrección de $\tau = 2$ errores.

In [32]:
y_corregida = D.decode_to_code(y)

Ahora verificamos si la palabra decodificada coincide con el polinomio generador del código.

In [33]:
y_corregida == C.generator_polynomial()

True

Vemos que **coinciden** y por tanto el mensaje recibido ha sido correctamente decodificado.